In [1]:
import os
import csv
import requests
from dotenv import load_dotenv

In [2]:
# Load environment variables from .env file
load_dotenv()

True

In [3]:
def get_total_users_by_location(locations):
    """
    Retrieves the total number of users on GitHub for each specified location.
    
    Args:
        locations (list): List of locations to retrieve user counts for.
        
    Returns:
        list: List of dictionaries, where each dictionary contains the total user count and the corresponding location.
    """
        
    users_by_location = []

    for location in locations:
        url = f'https://api.github.com/search/users?q=location:{location}'
        headers = {'Accept': 'application/vnd.github.v3+json'}
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            total_count = response.json().get('total_count')
            users_by_location.append({'total_users': total_count, 'location': location})
        else:
            print(f'Location request error "{location}": {response.status_code}')

    return users_by_location

# List of locations to retrieve user counts for
locations = [
    'Mozambique', 
    'Moçambique', 
    'Mocambique', 
    'Maputo', 
    'Matola',
#     'Inhambane', 
#     'Manica',
#     'Sofala',
#     'Tete',
#     'Zambezia',
#     'Nampula',
#     'Niassa',
#     'Cabo Delgado'
]

results = get_total_users_by_location(locations)

# Display the results
for result in results:
    print(f"{result['location']}: {result['total_users']}")


Mozambique: 1406
Moçambique: 256
Mocambique: 44
Maputo: 1065
Matola: 64


In [4]:
def get_users_ids_by_location(location):
    """
    Retrieve user IDs from GitHub API based on location.
    
    Args:
        location (str): Location of the users to search for.
    
    Returns:
        list: List of user IDs found.
    """
        
    url = f"https://api.github.com/search/users?q=location:{location}"
    token = os.getenv('GITHUB_TOKEN')
    headers = {'Authorization': f'Token {token}', 'Accept': 'application/vnd.github.v3+json'}

    params = {
        'per_page': 100,  # Number of items per page
        'page': 1,  # Initial page
        'order': 'asc',
        'sort': 'joined',
    }

    user_ids = []

    while True:
        response = requests.get(url, headers=headers, params=params)

        if response.status_code == 200:
            data = response.json()
            users = data["items"]
            user_ids.extend([user["id"] for user in users])

            if 'next' in response.links:
                url = response.links['next']['url']  # Next page of results
                params = {}  # No longer need the page parameters
            else:
                break
        else:
            print('Request Error:', response.status_code)
            break

    return user_ids


In [5]:
def write_csv(ids, file_name):
    """
    Write a list of IDs to a CSV file.

    Args:
        ids (list): List of IDs to be written to the CSV file.
        file_name (str): Name of the CSV file to be created or overwritten.

    Returns:
        None
    """

    data = [{'id': num} for num in ids]

    # Column names for the CSV file
    fields = ['id']

    # Write the data to the CSV file
    with open(file_name, 'w', newline='') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=fields)
        writer.writeheader()  # Write the column names to the file
        writer.writerows(data)  # Write the data to the file

    print(f'The IDs were successfully written to the file {file_name}!')


In [7]:
dir_name = os.getenv('DIR_NAME')
base_path = f'../../data/raw/{dir_name}'

os.makedirs(base_path, exist_ok=True)

for loc in locations:
    final_data = get_users_ids_by_location(loc)  

    file_name = f'{base_path}/{loc.lower()}.csv'
    
    write_csv(final_data, file_name)


The IDs were successfully written to the file ../../data/raw/2024_05/mozambique.csv!
The IDs were successfully written to the file ../../data/raw/2024_05/moçambique.csv!
The IDs were successfully written to the file ../../data/raw/2024_05/mocambique.csv!
The IDs were successfully written to the file ../../data/raw/2024_05/maputo.csv!
The IDs were successfully written to the file ../../data/raw/2024_05/matola.csv!
